#  Dokumentacja projektu Sentiment Classification
#### Aleksander Ogonowski
   
#### Paweł Rybak

#### Kornel Szymczyk 267778

##### 10 stycznia 2019 

## 1. Instrukcja użytkownika

Pakiet pszt zawiera dwa moduły:
- refactor_csv.py zawierający funkcje pomocnicze do refaktoryzacji zbioru danych Tweetów dotyczących produktów Apple,
- net.py zawierający klasę MLP, która jest definicją modelu sieci neuronowej.

### 1.1. Wymagania uruchomienia
- Python 3.6+
- Numpy

### 1.2. Przekształcenie zbioru danych wykorzystując moduł refactor_csv.py

Przed przystąpieniem do trenowania sieci neuronowej należy przekrzstałcić odpowiednio zbiór danych i zapisać go w nowym pliku csv. W tym celu korzystamy z funkcji *refactor(csv_path, new_csv_path)*. Pierwszy argument funkcji to ścieżka do pliku ze zbiorem danych w formacie *.csv*, drugi argument to ścieżka pliku, w którym zostanie zapisany przekształcony zbiór. 

Przykład kodu:

In [1]:
from pszt import refactor_csv

# File path to the csv file.
csv_path = 'Apple-Twitter-Sentiment-DFE.csv'

# File path to the new csv file.
new_csv_path = 'apple-twitter_example.csv'
    
refactor_csv.refactor(csv_path, new_csv_path)

Posiadając przekształcony zbiór plik csv należy przystąpić do wektoryzacji zbioru, w tym celu należy skorzystać z funkcji *vectorize_dataset(csv_path, x_train_path, y_train_path, ignore_words=None)*. Jej celem jest wykonanie przekształcenia zdań do modelu bag-of-words oraz zamiana sentymentu zdań (1, 3, 5) do postaci one-hot. Przekształcone zdania zapisywane są w tablicy numpy *x_train*, etykiety w tablicy numpy *y_train* oraz zapisywane są na dysku w formacie *.npy*. Pierwszy argument funkcji to ścieżka do pliku csv utworzonego przy pomocy funkcji *refactor()*, drugi argument to ścieżka pliku do zapisania tablicy *x_train*, trzeci argument to ścieżka pliku do zapisania tablicy *y_train* oraz ostatni argument przyjmuje opcjonalną tablicę słów, które mają być pomijane w zdaniach.

Przykład kodu:

In [2]:
refactor_csv.vectorize_dataset(new_csv_path, 'x_train_no_ignore_no_norm', 'y_train_no_ignore_no_norm')

In [3]:
import numpy as np
x_train = np.load('x_train_no_ignore_no_norm.npy')
np.max(x_train)

5.0

### 1.3. Trenowanie sieci neuronowej wykorzystując moduł net.py

W celu wytrenowania sieci tworzymy obiekt klasy MLP. Następnie dodajemy warstwy sieci wykorzystując metodę *add_layer(input_dim, output_dim, activation)*. Pierwszy argument określa liczbę wejść do warstwy, drugi argument określa liczbę wyjść z warstwy, a ostatni określa rodzaj funkcji aktywacji ('relu', 'sigmoid', 'softmax'). Ze względu na to, że etykiety zbioru reprezentowane są w postaci one-hot, ostatnia warstwa powinna zawsze posiadać output_dim=3 oraz activation='softmax'. Model sieci został napisany w taki sposób, że należy pominąć dodawanie warstwy wejściowej sieci neurnowej, a input_dim pierwszej warstwy powinien odpowiadać długości wektora danych wejściowych. Po dodaniu warstw należy je zainicjalizować funkcją *init_layers()*.

Proces trenowania wywołuje się funkcją *train(x, y_true, epochs, silent=False)*. Pierwszy argument to tablica numpy ze zdaniami w postaci bag-of-words, drugi argument to tablicy etykiet zdań w postaci one-hot, trzeci argument określa liczbę kroków trenowania, czwarty agrument opcjonalny określający czy wypisywać na ekran aktualny status trenowania sieci.

Przykład kodu:

In [3]:
import numpy as np
from pszt import net

# Create a neural net.
mlp = net.MLP();

# Load dataset.
x_train = np.load('x_train_3k.npy')
y_train = np.load('y_train_3k.npy')

# Add layers.
mlp.add_layer(input_dim=x_train.shape[1], output_dim=20, activation="sigmoid")
mlp.add_layer(input_dim=20, output_dim=3, activation="softmax")

# Initialize weights in layers.
mlp.init_layers()

# Train the neural net.
mlp.train(x=x_train, y_true=y_train, epochs=5)

loss: 3938.32, accuracy: 56.8%
loss: 8875.00, accuracy: 56.8%
loss: 23038.32, accuracy: 32.0%
loss: 6041.75, accuracy: 56.8%
loss: 4177.64, accuracy: 56.8%


Klasa MLP zawiera również metodę *k_fold_validation(x, y_true, k, epochs)*, która poddaje sieć neuronową walidacji k-fold.

Przykład kodu:

In [4]:
mlp.k_fold_validation(x=x_train, y_true=y_train, k=5, epochs=5)

Current fold: 0, Len of fold: 761
Current fold: 1, Len of fold: 761
Current fold: 2, Len of fold: 761
Current fold: 3, Len of fold: 761
Accuracies on k_folds: [57.763157894736835, 57.763157894736835, 57.763157894736835, 57.763157894736835]
Mean of accuracies: 57.763157894736835


## 2. Struktura programu

Najważniejszym elementem programu jest klasa MLP reprezentująca sieć neuronową. Tworzy ona pustą sieć. Należy do niej dodać warstwy przy pomocy metody *add_layer*. Dodaje ona jedynie informacje o nowej warstwie do metadanych sieci. Aby sieć faktycznie wykorzystała dodaną warstwę należy wywołać metodę *init_layers*. Dla każdej warstwy opisanej w strukturze sieci utworzy ona odpowiadającą macierz wag i przesunięć. Wartości dla wag warstw ukrytych są losowane z rozkładu jednostajnego. Przedstawia to poniższy fragment kodu:

In [ ]:
self.param_values['w' + str(i)] = np.random.uniform(
    low=(-1/np.sqrt(input_size)),
    high=(1/np.sqrt(input_size)),
    size=(input_size, output_size)
)
self.param_values['b' + str(i)] = np.random.uniform(
    low=(-1/np.sqrt(input_size)),
    high=(1/np.sqrt(input_size)),
    size=output_size
)

Wagi warstwy wyjściowej inicjowane są wartością 0.

Klasa ta posiada również funkcję *_forward(x)*, która wylicza wektor wyjściowy funkcji, w zależności od podanego wektora wejściowego. Jej implementacja jest trywialna. Dla każdej warstwy sieci wylicza wyjście i funkcję aktywacji, która następnie jest podawane jako wejście do kolejnej warstwy. Każda z pośrednich wartości takiego przejścia jest zapisywana w pamięci sieci. Jest to potrzebne do późniejszej wstecznej propagacji w procesie uczenia sieci. Wspomniana wsteczna propagacja została zaimplementowana w funkcji *_backward*. Funkcja ta wylicza gradient funkcji kosztu względem wag sieci. Wyliczone gradienty są zapisywane wewnątrz sieci, aby potem mogły być użyte wewnątrz funkcji *_update_weights*. Modyfikuje ona wagi sieci o wartości proporcjonalne do gradientu.

Uczenie sieci można zainicjować przy pomocy metody *train*, której najważniejszy fragment został przytoczony poniżej.

In [ ]:
def train(self, x, y_true, epochs, silent=False):
    for i in range(1, epochs+1):
        # Perform forward propagation over neural network.
        self._forward(x)

        # Perform backward propagation over neural network.
        self._backward(x, y_true)

        # Update weights of neural network.
        self._update_weights()

Parametrami tej funkcji jest wektor wartości wejściowych, oraz odpowiadającym im wartościom wyjściowym. Wymagane jest również podanie liczby epok, definiującej jak długo sieć ma się uczyć. Jest to jedyny zaimplementowany warunek stopu.
Uczenie odbywa się na całym zbiorze trenującym podanym do funkcji.

## 3. Kluczowe decyzje projektowe.



## 4. Wnioski dotyczące osiągniętych rezultatów